En este cuarderno optimizo el modelo elegido (XGBoost) y realizo el entrenamiento y la predicción con la combinación de valores de hiperparámetros recomendada por la optimización. En primer lugar importo alguna de las librerías necesarias:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split

Cargo el dataset que ofrece la competición (esta forma de de subirlo se debe a que este es un notebook de Google Colaboratory).

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
import io

kobe_df = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')))

Elimino aquellas variables que no voy a usar:

In [ ]:
kobe_df.drop(['game_event_id','game_id','team_id','team_name','game_date','lat', 'lon'], axis=1,
                     inplace=True)

Creo una variable que indica si los registros de la variables shot_made_flag son NaNs o no NaNs. Con esta variable podré seleccionar posteriormente el conjunto de test y el conjunto de entrenamiento:

In [ ]:
shot_nan = kobe_df['shot_made_flag'].isnull()

### TRANSFORMACIÓN DE VARIABLES

Transformo las variables seleccionadas para que reflejen lo observado en el análisis exploratorio. Realizo las mismas transformaciones que en el cuaderno "SELECCIÓN DE CARACTERÍSTICAS Y DE MODELO".

Transformo shot_distance de manera que aquellos valores superiores a 45 pies entreN dentro de un mismo valor (45), debido a que existen pocos registros superiores a este valor:

In [ ]:
kobe_df['shot_distance'] = kobe_df.apply(lambda row: row['shot_distance']  if row['shot_distance']<45\
                                             else 45, axis=1)

Creo una nueva variable, donde se refleje  si el equipo de Kobe era visitante (@/0) o residente (vs/1), ya que estos datos parecen ser relevantes según el análisis exploratorio:

In [ ]:
kobe_df['transformed_matchup'] = kobe_df.apply(lambda row: 0 if '@' in row['matchup'] else 1, axis=1 )

In [ ]:
kobe_df = kobe_df.drop('matchup', axis=1) #Borro 'matchup' ya que no me será de más relevancia.

Transformo la variable action_type. Para ello, primero creo una variable donde aparece la ocurrencia de cada categoría de action_type en el dataset:

In [ ]:
action_count = kobe_df['action_type'].value_counts()

Posteriormente, modifico la variable para que aparezcan las categorías de action_type con registros en el dataset superiores a 150, mientras que aquellas categorías con menos de 150 registros serán sustituidas por una categoría global llamada 'other'. Utilizo en el proceso la variable 'action_counts', antes definida:

In [ ]:
kobe_df['action_type'] = kobe_df.apply(lambda row:row['action_type'] if action_count[row['action_type']]>150\
                                             else 'other', axis=1)

Ya que minutes_remaining se refiere a los últimos 11 minutos del periodo y seconds_remaining al último periodo de este, creo una variable donde se resuma y condense esta información, donde aparezcan los doce minutos del último periodo:

In [ ]:
kobe_df['time_remaining'] = kobe_df.apply(lambda row: row['minutes_remaining'] * 60 + row['seconds_remaining'], axis=1)

### SELECCIÓN DE VARIABLES

Aplico la selección ya llevada a cabo en el cuaderno "SELECCIÓN DE CARACTERÍSTICAS Y DE MODELO":

In [ ]:
kobe_df = kobe_df[['action_type', 'combined_shot_type','loc_y', 'season', 'shot_distance', 
                'shot_zone_area','shot_zone_range', 'transformed_matchup','shot_id',  #incluyo shot_id porque posteriormente se usará para identificar los disparos del
                'time_remaining','shot_made_flag']]                                   #conjunto de test.

Dumifico las variables categóricas. Así, tal y como dice el trabajo escrito, obtendré mejores resultados que con la codificación nominal

In [ ]:
features = ['action_type', 'combined_shot_type', 'season', 
            'shot_zone_area','shot_zone_range', 'transformed_matchup'] # Lista de variables categóricas.

kobe = kobe_df[['loc_y','shot_distance','time_remaining']] # Dataset con las variables cuantitativas

for f in features: #Bucle que dumifica cada variable categórica y las une con las variables cuantitativas.
    
    kobe = pd.concat([kobe,pd.get_dummies(kobe_df[f], prefix=f)],axis=1)

### OPTIMIZACIÓN

La optimización se llevará a cabo mediante optimización Random Search.

Selecciono los datos de **x** y los datos de** y** que se utilizarán en el proceso de optimización. Para ello selecciono los datos del conjunto de entrenamiento con shot_nan:

In [ ]:
x = kobe[~shot_nan] # Registros correspondientes a los valores no nulos de la variable shot_made_flag
y = kobe_df['shot_made_flag'][~shot_nan] # Registros correspondientes a los valores no nulos de la variable shot_made_flag.

Llamo al clasificador de XGBoost:

In [ ]:
xgb = xgboost.XGBClassifier() 

Divido entre train y validation con train_test_split (de scikit learn) para evitar sobreaprendizaje en proceso de optimización:

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.1, random_state=1) #validation tendrá el 10% de los datos (test_size=1).

Establezco el espacio de  búsqueda y optimizo con Random Search, usando como estimador el xgboost:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV #Importamos RandomizedSearchCV de Scikit learn, el optimizador Random Search de esta librería.
from sklearn.metrics import accuracy_score

#El espacio de búsqueda de la optimización:
param= {'booster':['gbtree', 'dart'], #El tipo de booster a usar.
        'max_depth': [6, 10, 15, 20], #Máxima profundidad de los árboles estimadores.
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3], 
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0], #Proporción de submuestra de las instancias de entrenamiento. Muestreo aleatorio antes de "cultivar" los árboles.
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], #Es la proporción de submuestras de columnas al construir cada árbol. El submuestreo ocurre una vez por cada árbol.
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], #Es la proporción de submuestra de columnas para cada nivel. El submuestreo ocurre una vez por cada nuevo nivel de profundidad alcanzado en un árbol.
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0], #Suma mínima de peso de instancia (hessian) necesaria en un hijo. Si el paso de partición del árbol da como resultado un nodo hoja con una suma de peso de instancia menor que min_child_weight, entonces el proceso de construcción abandonará la partición adicional.
        'gamma': [0, 0.25, 0.5, 1.0], #Reducción de pérdida mínima requerida para realizar una partición adicional en un nodo hoja del árbol. Cuanto mayor sea la gamma, más conservador será el algoritmo.
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0], #Término de regularización L2 sobre los pesos. Incrementar este valor hará el modelo más conservativo.
        'n_estimators':[6, 20, 100, 150, 200, 400]} #nº de estimadores o árboles.



random_class= RandomizedSearchCV(estimator = xgb, param_distributions = param, 
                                 n_iter = 180, cv = 5, random_state=0, n_jobs = -1) #Establezco los parámetros de la optimización


eval_set=[(x_val,y_val)] #Llevar a cabo este paso es necesario para aplicar el early_stopping en la optimización, que evitará el sobreaprendizaje.

#Ejecuto la optimización con el early stopping:

random_class.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=5, eval_metric='logloss', verbose=True) # early_stopping_round =5 ==>Si no se observa mejora se para en 5 vueltas/iteraciones.

best_estimator= random_class.best_estimator_ # La mejor combinación de hiperparámetros hallada por la optimización.

best_estimator

[0]	validation_0-logloss:0.679974
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.669441
[2]	validation_0-logloss:0.660785
[3]	validation_0-logloss:0.654024
[4]	validation_0-logloss:0.647291
[5]	validation_0-logloss:0.641822
[6]	validation_0-logloss:0.637202
[7]	validation_0-logloss:0.633351
[8]	validation_0-logloss:0.630141
[9]	validation_0-logloss:0.627066
[10]	validation_0-logloss:0.624025
[11]	validation_0-logloss:0.622041
[12]	validation_0-logloss:0.620171
[13]	validation_0-logloss:0.618345
[14]	validation_0-logloss:0.616783
[15]	validation_0-logloss:0.615596
[16]	validation_0-logloss:0.614265
[17]	validation_0-logloss:0.613005
[18]	validation_0-logloss:0.612215
[19]	validation_0-logloss:0.611258
[20]	validation_0-logloss:0.610511
[21]	validation_0-logloss:0.609837
[22]	validation_0-logloss:0.609065
[23]	validation_0-logloss:0.608365
[24]	validation_0-logloss:0.607986
[25]	validation_0-logloss:0.607708
[26]	validation_0-logloss:0.60736

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=0.4,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=0.5, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=50.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [ ]:
best_estimator #Aquí vemos los hiperparámetros del mejor estimador hallado por la optimización de Random Search.

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=0.4,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=0.5, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=50.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

### ENTRENAMIENTO Y PREDICCIÓN

Con el mejor estimador hallado en la optimización entreno, predigo y envío los resultados a la competición:

In [ ]:
eval_set=[(x_val,y_val)]
best_estimator.fit(x_train, y_train, eval_set=eval_set, early_stopping_rounds=5, eval_metric='logloss', verbose=True) #Con el mejor estimador entreno.

[0]	validation_0-logloss:0.679974
Will train until validation_0-logloss hasn't improved in 5 rounds.
[1]	validation_0-logloss:0.669441
[2]	validation_0-logloss:0.660785
[3]	validation_0-logloss:0.654024
[4]	validation_0-logloss:0.647291
[5]	validation_0-logloss:0.641822
[6]	validation_0-logloss:0.637202
[7]	validation_0-logloss:0.633351
[8]	validation_0-logloss:0.630141
[9]	validation_0-logloss:0.627066
[10]	validation_0-logloss:0.624025
[11]	validation_0-logloss:0.622041
[12]	validation_0-logloss:0.620171
[13]	validation_0-logloss:0.618345
[14]	validation_0-logloss:0.616783
[15]	validation_0-logloss:0.615596
[16]	validation_0-logloss:0.614265
[17]	validation_0-logloss:0.613005
[18]	validation_0-logloss:0.612215
[19]	validation_0-logloss:0.611258
[20]	validation_0-logloss:0.610511
[21]	validation_0-logloss:0.609837
[22]	validation_0-logloss:0.609065
[23]	validation_0-logloss:0.608365
[24]	validation_0-logloss:0.607986
[25]	validation_0-logloss:0.607708
[26]	validation_0-logloss:0.60736

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=0.4,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=0.5, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=50.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

Predigo con el conjunto de test marcado en la competición:


In [ ]:
test = kobe[shot_nan] # Registros correspondientes a valores nulos de la variable shot_made_flag.
pred = best_estimator.predict_proba(test)[:,1] # Predigo la probabilidad de que un caso sea éxito de tiro a canasta
                                               # en términos de probabilidad.
shot_id = kobe_df['shot_id'][shot_nan]
submission = pd.DataFrame({"shot_id":shot_id, "shot_made_flag":pred}) # Creo el dataset que se enviará a la competición.
submission.sort_values('shot_id',  inplace=True) # Ordeno por id del tiro para que la competición de Kaggle pueda evaluar el modelo correctamente.

### PREPARACIÓN DEL RESULTADO DE LA CLASIFICACIÓN PARA ENVIAR

Preparo el archivo que será enviado a la competición:

In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


Creo el archivo csv que enviaré a la competición:

In [ ]:
submission.to_csv("file_xgboost_sinlocx.csv",index=False) # Este modelo da un log loss=0.60365, puesto superior al 301 de 1117 en la competición.
!cp file_xgboost_sinlocx.csv "drive/My Drive/" #guardo la predicción realizada en un csv para enviarlo a la competición.